## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images


In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')
# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)
        
cv2.destroyAllWindows()


## Image Undistortion

In [54]:
def cal_undistort(img, objpoints, imgpoints):
    ret, mtx, dist, revcs, tves = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1:], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist, mtx, dist;

for fname in images:
    img = cv2.imread(fname)
    undist, mtx, dist = cal_undistort(img, objpoints, imgpoints)
    cv2.imshow('undistorted_image',undist)
    cv2.waitKey(50)
cv2.destroyAllWindows()

image1 = cv2.imread('camera_cal/calibration1.jpg')
undist, mtx, dist = cal_undistort(image1, objpoints, imgpoints)
cv2.imwrite('output_images/uncalib_image.jpg',image1)
cv2.imwrite('output_images/undist_image.jpg',undist)

True

## Generate binary thresholded image

In [60]:
#image = cv2.imread('test_images/straight_lines1.jpg')
image = cv2.imread('test_images/test4.jpg')
def thresholded(img,s_thresh=(120,255),sx_thresh=(40,150)): 
    img = np.copy(img)
    undist, mtx, dist = cal_undistort(img, objpoints, imgpoints)
    #cv2.imwrite('output_images/corrected_image.jpg',undist)
    #hls
    hls = cv2.cvtColor(undist, cv2.COLOR_RGB2HLS)
    s_channel = hls[: ,:, 2]
    l_channel = hls[: ,:, 1]
    #gray
    gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)
    
    #sobel X
    sobelx = cv2.Sobel(gray, cv2. CV_64F, 1, 0, ksize=3)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobelx = np.uint8(255*abs_sobelx /np.max(abs_sobelx))
    #threshold X
    sxbinary = np.zeros_like(scaled_sobelx)
    sxbinary[(scaled_sobelx >= sx_thresh[0]) & (scaled_sobelx <= sx_thresh[1])] = 1

    #threshols s&l
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= s_thresh[0]) & (l_channel <= s_thresh[1])] = 1
    
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[((s_binary == 1)&(l_binary == 1))|(sxbinary == 1)] = 1

    return combined_binary
thresh_img = thresholded(image)
plt.imshow(thresh_img, cmap='gray')

cv2.imwrite('output_images/thresh_image.jpg',thresh_img)


True

In [124]:
#block use to select vertices
#image = cv2.imread('test_images/test4.jpg')
plt.imshow(thresh_img,cmap='gray')
print((img.shape[1], img.shape[0]))
plt.plot(280,img.shape[0],'.')
plt.plot(1120,img.shape[0],'.')
plt.plot(590,460,'.')
plt.plot(720,460,'.')


(1280, 720)


## Transpose image to create a bird-eye view

In [56]:
def unwarp(img,mtx,dist):
    img_size = (img.shape[1], img.shape[0])
    #ret, mtx, dist, revcs, tves = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1:], None, None)    
    #undist = cv2.undistort(img, mtx, dist, None, mtx)
    src = np.float32(
        [[280, img.shape[0]], #bottom left
        [1120, img.shape[0]], #bottom right
        [590, 460], #top left
        [720, 460] #top right
        ])
    dst = np.float32(
        [[340, img.shape[0]],
        [1060, img.shape[0]], 
        [340, 0],
        [1060, 0]
        ])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped, dst, src
unwarp_img, dst, src = unwarp(thresh_img, mtx, dist)
plt.imshow(unwarp_img,cmap='gray')
plt.figure()
unwarp_img_RGB, dst, src = unwarp(image, mtx, dist)
plt.imshow(unwarp_img_RGB)

cv2.imwrite('output_images/unwarp_image.jpg',unwarp_img_RGB)
'''original1 = cv2.line(image,(280,1280),(590,460),(255,0,0),2)
original2 = cv2.line(original1,(1120,1280),(720,460),(255,0,0),2)
original3 = cv2.line(original2,(590,460),(720,460),(255,0,0),2)
cv2.imshow('unwarp image',original3)
warped1 = cv2.line(unwarp_img_RGB,(280,1120),(280,0),(255,0,0),2)
warped2 = cv2.line(warped1,(1060,1280),(1060,0),(255,0,0),2)
cv2.imshow('warped image',warped2)
fig, (ax1, ax2) = plt.subplots(1,2)
ax1.imshow(image)
ax2.imshow(unwarp_img)
cv2.imshow('ori',image)
cv2.imshow('img',unwarp_img)'''

"original1 = cv2.line(image,(280,1280),(590,460),(255,0,0),2)\noriginal2 = cv2.line(original1,(1120,1280),(720,460),(255,0,0),2)\noriginal3 = cv2.line(original2,(590,460),(720,460),(255,0,0),2)\ncv2.imshow('unwarp image',original3)\nwarped1 = cv2.line(unwarp_img_RGB,(280,1120),(280,0),(255,0,0),2)\nwarped2 = cv2.line(warped1,(1060,1280),(1060,0),(255,0,0),2)\ncv2.imshow('warped image',warped2)\nfig, (ax1, ax2) = plt.subplots(1,2)\nax1.imshow(image)\nax2.imshow(unwarp_img)\ncv2.imshow('ori',image)\ncv2.imshow('img',unwarp_img)"

## Lane finding using sliding windows

In [50]:
#sliding windows
def find_lane_pixels(img):
    histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
    out_img = np.dstack((img, img, img))
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    nwindows = 15
    margin = 50
    minpix = 50

    window_height = np.int(img.shape[0]//nwindows)
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    leftx_current = leftx_base
    rightx_current = rightx_base

    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin 
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        pass

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def fit_polynomial(img):
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(img)
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.plot(mid_fitx, ploty, color='yellow')
    
    return left_fit, right_fit, out_img

left_fit, right_fit, out_img = fit_polynomial(unwarp_img)

plt.imshow(out_img)

## Lane finding based on previous sliding window function

In [57]:
#search form prior
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

def search_around_poly(img):
    margin = 100
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(img.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((img, img, img))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    left_line = np.array([np.transpose(np.vstack([left_fitx, 
                              ploty]))])
    right_line = np.array([np.flipud(np.transpose(np.vstack([right_fitx, 
                              ploty])))])
    fill = np.hstack((left_line, right_line))
    cv2.fillPoly(window_img, np.int_([fill]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    
    return result,ploty, leftx, rightx, lefty, righty, left_fitx, right_fitx

result,ploty, leftx, rightx, lefty, righty,left_fitx, right_fitx = search_around_poly(unwarp_img)

plt.imshow(result)
cv2.imwrite('output_images/fit_image.jpg',result)

True

## Calculate lane curvature

In [35]:
def curvature(pts1,pts2,pts3,pts4,pts5,pts6,pts7,img):
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    left_fit_cr = np.polyfit(pts3*ym_per_pix, pts1*xm_per_pix, 2)
    right_fit_cr = np.polyfit(pts4*ym_per_pix, pts2*xm_per_pix, 2)

    y_eval = np.max(pts5)
    left_cur =((1 + (2*left_fit_cr[0]*y_eval*xm_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_cur =((1 + (2*right_fit_cr[0]*y_eval*xm_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    rad = np.mean([left_cur,right_cur])
    
    lane_center =np.mean([pts6[0],pts7[0]])
    car_center = img.shape[1]//2
    dis_from_center = np.absolute(lane_center-car_center)*xm_per_pix
    
    if lane_center > car_center:
        status = 'left from center'
    if lane_center < car_center:
        status = 'right from center'
    if lane_center == car_center:
        status = 'from center'
    
    text = ['Radius of curvature = {0:.0f}(m)'.format(rad),'Vehicle is {0:0.2f} '.format(dis_from_center)+ status]
    
    return text

output= curvature(leftx,rightx,lefty,righty,ploty,left_fitx,right_fitx,result)

print(output)



['Radius of curvature = 975(m)', 'Vehicle is 0.56 left from center']


## Inverse transpose image

In [58]:
def warp(text, linefit_img, original_img):
    img_size = (linefit_img.shape[1], linefit_img.shape[0])    
    invM = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(linefit_img, invM, img_size, flags=cv2.INTER_LINEAR)
    
    overlay = cv2.addWeighted(original_img,1,warped,1,0)
    font     = cv2.FONT_HERSHEY_SIMPLEX
    #wrapped_text = ['Radius of curvature = {0:.0f}(m)'.format(rad),'Vehicle is {0:0.2f} '.format(dis_from_center)+ status]
    x,y = 300,35
    font_size= 1
    font_color= (255,255,255)
    font_thickness= 2 

    i = 0
    for line in text:
        textsize = cv2.getTextSize(line, font, font_size, font_thickness)[0]
        gap = textsize[1] + 5
        y = 35 + i * gap
        x = 300 
        text_img=cv2.putText(overlay, line, (x, y), font,
                            font_size, 
                            font_color, 
                            font_thickness, 
                            lineType = cv2.LINE_AA)
        i +=1
    
    return text_img
output_img = warp(output,result, image)

plt.imshow(output_img)
#plt.imshow(warp_img, alpha=0.5)
#overlay = cv2.addWeighted(image,1,warp_img,1,0)
#cv2.imshow('',overlay)
cv2.imwrite('output_images/processed_image.jpg',output_img)

True

## Pipeline to process images

In [44]:
#image testing pipeline 
def pipeline(img):
    thresh_img = thresholded(img)
    unwarp_img, dst, src = unwarp(thresh_img, mtx, dist)
    left_fit, right_fit, out_img = fit_polynomial(unwarp_img)
    result,ploty, leftx, rightx, lefty, righty,left_fitx, right_fitx = search_around_poly(unwarp_img)
    output= curvature(leftx,rightx,lefty,righty,ploty,left_fitx,right_fitx,result)
    processed = warp(output,result, img) 
    return processed

input = cv2.imread('test_images/test6.jpg')

out = pipeline(input)
cv2.imshow('',out)


In [47]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
white_output = 'Processed_project_video.mp4'
#clip1 = VideoFileClip("project_video.mp4").subclip(0,2)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(pipeline) #need to add function here!
%time white_clip.write_videofile(white_output, audio=False)

Moviepy - Building video Processed_project_video.mp4.
Moviepy - Writing video Processed_project_video.mp4



Moviepy - Done !
Moviepy - video ready Processed_project_video.mp4
Wall time: 32min 17s


In [48]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [51]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
white_output_1 = 'Processed_challenge_video.mp4'
#clip1 = VideoFileClip("project_video.mp4").subclip(0,2)
clip1 = VideoFileClip("challenge_video.mp4")
white_clip_1 = clip1.fl_image(pipeline) #need to add function here!
%time white_clip.write_videofile(white_output_1, audio=False)

t:   0%|                                                                             | 0/485 [00:00<?, ?it/s, now=None]

Moviepy - Building video Processed_challenge_video.mp4.
Moviepy - Writing video Processed_challenge_video.mp4




t:  14%|█████████▍                                                          | 67/485 [01:38<10:14,  1.47s/it, now=None]


t:  27%|██████████████████▎                                                | 133/485 [03:16<08:41,  1.48s/it, now=None]


t:  28%|███████████████████                                                | 138/485 [03:23<08:35,  1.48s/it, now=None]

TypeError: expected non-empty vector for x

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [53]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
white_output_2 = 'Processed_harder_challenge_video.mp4'
#clip1 = VideoFileClip("project_video.mp4").subclip(0,2)
clip2 = VideoFileClip("harder_challenge_video.mp4")
white_clip_2 = clip2.fl_image(pipeline) #need to add function here!
%time white_clip.write_videofile(white_output_2, audio=False)

                                                                                                                       
t:  28%|███████████████████                                                | 138/485 [12:53<08:36,  1.49s/it, now=None]
                                                                                                                       
t:  28%|███████████████████                                                | 138/485 [05:16<08:35,  1.48s/it, now=None]

Moviepy - Building video Processed_harder_challenge_video.mp4.
Moviepy - Writing video Processed_harder_challenge_video.mp4





t:   0%|                                                                             | 0/485 [00:00<?, ?it/s, now=None]

t:   0%|▎                                                                    | 2/485 [00:02<08:25,  1.05s/it, now=None]

t:   1%|▍                                                                    | 3/485 [00:03<09:22,  1.17s/it, now=None]

t:   1%|▌                                                                    | 4/485 [00:04<09:57,  1.24s/it, now=None]

t:   1%|▋                                                                    | 5/485 [00:06<10:24,  1.30s/it, now=None]

t:   1%|▊                                                                    | 6/485 [00:07<10:40,  1.34s/it, now=None]

t:   1%|▉                                                                    | 7/485 [00:09<10:51,  1.36s/it, now=None]

t:   2%|█▏                                                                   | 8/485 [00:10<11:05,  1.40s/it, now=None]

t:   2%|█▎                    

t:  13%|█████████                                                           | 65/485 [01:35<10:26,  1.49s/it, now=None]

t:  14%|█████████▎                                                          | 66/485 [01:37<10:18,  1.48s/it, now=None]

t:  14%|█████████▍                                                          | 67/485 [01:38<10:19,  1.48s/it, now=None]

t:  14%|█████████▌                                                          | 68/485 [01:40<10:14,  1.47s/it, now=None]

t:  14%|█████████▋                                                          | 69/485 [01:41<10:17,  1.48s/it, now=None]

t:  14%|█████████▊                                                          | 70/485 [01:43<10:17,  1.49s/it, now=None]

t:  15%|█████████▉                                                          | 71/485 [01:44<10:18,  1.49s/it, now=None]

t:  15%|██████████                                                          | 72/485 [01:46<10:19,  1.50s/it, now=None]

t:  15%|██████████▏             

t:  27%|█████████████████▊                                                 | 129/485 [03:07<08:17,  1.40s/it, now=None]

t:  27%|█████████████████▉                                                 | 130/485 [03:08<08:16,  1.40s/it, now=None]

t:  27%|██████████████████                                                 | 131/485 [03:09<08:15,  1.40s/it, now=None]

t:  27%|██████████████████▏                                                | 132/485 [03:11<08:13,  1.40s/it, now=None]

t:  27%|██████████████████▎                                                | 133/485 [03:12<08:12,  1.40s/it, now=None]

t:  28%|██████████████████▌                                                | 134/485 [03:14<08:11,  1.40s/it, now=None]

t:  28%|██████████████████▋                                                | 135/485 [03:15<08:08,  1.40s/it, now=None]

t:  28%|██████████████████▊                                                | 136/485 [03:17<08:23,  1.44s/it, now=None]

t:  28%|██████████████████▉     

TypeError: expected non-empty vector for x

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output_2))